In [1]:
import nltk
#import ast
#import re
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.corpus import sentiwordnet
import pandas as pd

In [2]:
df=pd.read_csv("reddit.csv")

In [3]:
df.describe()

,comms_num,created,score,view_num
count,11898.000000,1.189800e+04,11898.000000,0.0
mean,72.514960,1.569871e+09,2573.083964,NaN
std,234.901785,1.860284e+06,7489.456335,NaN
min,0.000000,1.560093e+09,0.000000,NaN
25%,4.000000,1.569287e+09,32.000000,NaN
50%,14.000000,1.570654e+09,237.000000,NaN
75%,61.000000,1.571094e+09,2238.750000,NaN
max,8141.000000,1.572103e+09,130960.000000,NaN


In [4]:
df1 = df.sample(n=1000)

In [5]:
df1.describe()


,comms_num,created,score,view_num
count,1000.000000,1.000000e+03,1000.000000,0.0
mean,62.210000,1.569885e+09,2184.850000,NaN
std,137.269315,1.900039e+06,5435.255468,NaN
min,0.000000,1.560093e+09,0.000000,NaN
25%,4.000000,1.569273e+09,31.750000,NaN
50%,14.000000,1.570704e+09,217.500000,NaN
75%,61.250000,1.571115e+09,2157.750000,NaN
max,1669.000000,1.572096e+09,64209.000000,NaN


In [6]:
text = ""
for index, row in df1.iterrows():
    comments = row['comment']
    try:
        if len(comments) != 0 :
            for i in comments: 
                text = text + i
    except:
        pass 

In [7]:

my_stop_words = ["//WWW.YOUTUBE.COM/WATCH ","A","[","]","%","HTTPS","’","U","T","’ S","% E6 %","] [ ] [","] [","HTTPS ]","“","”","[ ]","’ T","% E5 %","[ ] [","[ [","I","S","HTTPS [","] ]",">","[ KONG"]



In [8]:
tokenizedReviews={}
tokenizer = nltk.tokenize.punkt.PunktSentenceTokenizer()
uniqueId=1;
cachedStopWords = nltk.corpus.stopwords.words("english")
my_stop_words = ["//WWW.YOUTUBE.COM/WATCH ","[","]","%","HTTPS","’","U","T","’ S","% E6 %"]
for i in my_stop_words:

    cachedStopWords.append(i) 

for sentence in tokenizer.tokenize(text):      
    tokenizedReviews[uniqueId]=sentence
    uniqueId+=1
    

   

In [ ]:
type(cachedStopWords)

In [22]:
#print(tokenizedReviews)

In [9]:
# !pip install contractions
#import contractions
outputPost={}
for key,value in tokenizedReviews.items():
    
    outputPost[key]=nltk.pos_tag(nltk.word_tokenize(value))
    

In [25]:
#print(outputPost)

In [10]:

prevWord=''
prevTag=''
currWord=''
aspectList=[]
outputDict={}
    
for key,value in outputPost.items():
    for word,tag in value:
    
        if(tag=='NN' or tag=='NNP'):
            if(prevTag=='NN' or prevTag=='NNP'):
                currWord= prevWord + ' ' + word
            else:
                if prevWord.upper() not in my_stop_words:
                    aspectList.append(prevWord.upper())
                    currWord= word
        prevWord=currWord
        prevTag=tag

#Eliminating aspect which has 1 or less count
for aspect in aspectList:
        if(aspectList.count(aspect)>100):
                if(outputDict.keys()!=aspect):
                        outputDict[aspect]=aspectList.count(aspect)
                        
outputAspect=sorted(outputDict.items(), key=lambda x: x[1],reverse = True)
   

In [27]:
#print(aspectList)
#print(outputAspect)

In [11]:
df = pd.DataFrame(columns=["aspect","positive","negative","neutral"])
df.head()

,aspect,positive,negative,neutral


In [12]:
def identifyOpinionWords(output, aspect):       
  
   
    outputAspectOpinionTuples={}
    orientationCache={}
    negativeWordSet = {"don't","never", "nothing", "nowhere", "noone", "none", "not",
                  "hasn't","hadn't","can't","couldn't","shouldn't","won't",
                  "wouldn't","don't","doesn't","didn't","isn't","aren't","ain't"}
    for aspect,no in aspect:
        aspectTokens= word_tokenize(aspect)
        count=0
        for key,value in output.items():
            condition=True
            isNegativeSen=False
            for subWord in aspectTokens:
                if(subWord in str(value).upper()):
                    condition = condition and True
                else:
                    condition = condition and False
            if(condition):
                for negWord in negativeWordSet:
                    if(not isNegativeSen):#once senetence is negative no need to check this condition again and again
                        if negWord.upper() in str(value).upper():
                            isNegativeSen=isNegativeSen or True
                outputAspectOpinionTuples.setdefault(aspect,[0,0,0])
                for word,tag in value:
                     if(tag=='JJ' or tag=='JJR' or tag=='JJS'or tag== 'RB' or tag== 'RBR'or tag== 'RBS'):
                            count+=1
                            if(word not in orientationCache):
                                orien=orientation(word)
                                orientationCache[word]=orien
                            else:
                                orien=orientationCache[word]
                            if(isNegativeSen and orien is not None):
                                orien= not orien
                            if(orien==True):
                                outputAspectOpinionTuples[aspect][0]+=1
                            elif(orien==False):
                                outputAspectOpinionTuples[aspect][1]+=1
                            elif(orien is None):
                                outputAspectOpinionTuples[aspect][2]+=1
        if(count>0):
            #print(aspect,' ', outputAspectOpinionTuples[aspect][0], ' ',outputAspectOpinionTuples[aspect][1], ' ',outputAspectOpinionTuples[aspect][2])
            outputAspectOpinionTuples[aspect][0]=outputAspectOpinionTuples[aspect][0]
            #outputAspectOpinionTuples[aspect][1]=round((outputAspectOpinionTuples[aspect][1]/count)*100,2)
            outputAspectOpinionTuples[aspect][1]=outputAspectOpinionTuples[aspect][1]
            outputAspectOpinionTuples[aspect][2]=outputAspectOpinionTuples[aspect][2]
            result = [aspect,outputAspectOpinionTuples[aspect][0],outputAspectOpinionTuples[aspect][1], outputAspectOpinionTuples[aspect][2]]
            #rs = pd.Series(result)
            df.loc[len(df)] = result
            print(aspect,':\t\tPositive => ', outputAspectOpinionTuples[aspect][0], '\tNegative => ',outputAspectOpinionTuples[aspect][1],"nerural =>",outputAspectOpinionTuples[aspect][2] )
        else:
            pass
            #print('count=0')
    return result 
#-----------------------------------------------------------------------------------
def orientation(inputWord): 
    wordSynset=wordnet.synsets(inputWord)
    if(len(wordSynset) != 0):
        word=wordSynset[0].name()
        orientation=sentiwordnet.senti_synset(word)
        #pol_score = sia.polarity_scores(word)
        #print(word, pol_score["compound"])
        if(orientation.pos_score() > orientation.neg_score()):
        #if(pol_score["compound"]>0.03):
          
            return True
        elif(orientation.pos_score() > orientation.neg_score()):
            
            return False

In [13]:
result = identifyOpinionWords(outputPost, outputAspect)

CHINA :		Positive =>  3211 	Negative =>  1584 nerural => 19264
HONG KONG :		Positive =>  2344 	Negative =>  1270 nerural => 16219
HK :		Positive =>  2480 	Negative =>  1253 nerural => 14586
POLICE :		Positive =>  2181 	Negative =>  1132 nerural => 13454
GOVERNMENT :		Positive =>  1578 	Negative =>  780 nerural => 9626
TIME :		Positive =>  1367 	Negative =>  794 nerural => 9161
WAY :		Positive =>  1981 	Negative =>  1126 nerural => 13284
WORLD :		Positive =>  1156 	Negative =>  597 nerural => 6838
COUNTRY :		Positive =>  813 	Negative =>  512 nerural => 5530
CCP :		Positive =>  982 	Negative =>  436 nerural => 5713
US :		Positive =>  13651 	Negative =>  6016 nerural => 64122
SOMETHING :		Positive =>  741 	Negative =>  369 nerural => 3996
FREEDOM :		Positive =>  673 	Negative =>  353 nerural => 4316
THING :		Positive =>  2733 	Negative =>  2234 nerural => 18318
POINT :		Positive =>  1034 	Negative =>  725 nerural => 6208
ANYTHING :		Positive =>  472 	Negative =>  420 nerural => 3671
LOT 

S :		Positive =>  29348 	Negative =>  10174 nerural => 151826
CULTURE :		Positive =>  165 	Negative =>  106 nerural => 1153
THAT\ :		Positive =>  162 	Negative =>  128 nerural => 956
DIFFERENCE :		Positive =>  155 	Negative =>  71 nerural => 971
SOURCE :		Positive =>  440 	Negative =>  230 nerural => 3499
PLEASE :		Positive =>  287 	Negative =>  155 nerural => 1579
NOBODY :		Positive =>  109 	Negative =>  43 nerural => 638
BUNCH :		Positive =>  159 	Negative =>  81 nerural => 720
MIND :		Positive =>  390 	Negative =>  179 nerural => 1981
LEVEL :		Positive =>  173 	Negative =>  123 nerural => 1157
CONTEXT :		Positive =>  148 	Negative =>  66 nerural => 933
KONG :		Positive =>  2362 	Negative =>  1273 nerural => 16357
CALIFORNIA :		Positive =>  161 	Negative =>  78 nerural => 983
STATEMENT :		Positive =>  170 	Negative =>  133 nerural => 1311
CHINESE :		Positive =>  1633 	Negative =>  752 nerural => 12067
RESPONSE :		Positive =>  162 	Negative =>  102 nerural => 865
LINE :		Positive =>  

In [15]:
df.head()

,aspect,positive,negative,neutral
0,CHINA,3211,1584,19264
1,HONG KONG,2344,1270,16219
2,HK,2480,1253,14586
3,POLICE,2181,1132,13454
4,GOVERNMENT,1578,780,9626


eyebowling and remove stop words 

In [25]:
df.drop(df.loc[df['aspect']=='I\\'].index, inplace=True)

In [39]:
df.drop(df.loc[df['aspect']=='NOTHING'].index, inplace=True)

In [42]:
df.drop(df.loc[df['aspect']=='THAT\\'].index, inplace=True)

In [40]:
df.head(50)

,aspect,positive,negative,neutral
0,CHINA,3211,1584,19264
1,HONG KONG,2344,1270,16219
2,HK,2480,1253,14586
3,POLICE,2181,1132,13454
4,GOVERNMENT,1578,780,9626
5,TIME,1367,794,9161
6,WAY,1981,1126,13284
7,WORLD,1156,597,6838
8,COUNTRY,813,512,5530
9,CCP,982,436,5713


In [41]:
df.to_csv("aspect.csv",index=False)